In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def get_data(table, year):
    df = pd.read_html(str(table), flavor='bs4')[0]
    states_df = pd.read_html(str(table), flavor='bs4', extract_links='body')[0]  # to get state from url

    # Rename columns
    columns = ['launch_datetime', 'rocket', '1', '2', '3', '4', '5', '6']
    df.columns = columns

    columns = ['launch_datetime', 'rocket_state', 'payload_state', '2', 'launch_site_state', '4', 'lsp_state', '6']
    states_df.columns = columns


    # Remove remarks and garbage lines
    filt = (
        ((df['1'] == df['2']) & (df['1'] == df['3']) & (df['1'] == df['4']) & (df['1'] == df['5']) & (df['1'] == df['6']))
        | (df['launch_datetime'] == '← Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec →')
        | (states_df['rocket_state'].str[1].isnull() & df['rocket'].isnull() & df['1'].isnull() & df['2'].isnull() & df['3'].isnull() & df['4'].isnull() & df['5'].isnull() & df['6'].isnull())
           )

    df = df[~filt]
    states_df = states_df.loc[~filt, ['rocket_state', 'payload_state', 'launch_site_state', 'lsp_state']]
    df_len = len(df)
    
    filt = states_df['rocket_state'].str[1].notnull() & df['rocket'].isnull()
    df.loc[filt, 'rocket'] = '-1'

    # Get states and join it to main df
    for column in states_df.columns:
        states_df[column + '_check'] = states_df[column].str[0]
        states_df[column] = states_df[column].str[1].str.replace('/wiki/', '').str.replace('_', ' ')
        states_df.loc[states_df[column].isin(['', 'N/A']), column] = np.nan
        states_df.loc[states_df[column + '_check'].isin(['', 'N/A']), column + '_check'] = np.nan

    df = df.join(states_df, how='left')

    check_filt = (
        (df['rocket'].fillna('-1') != df['rocket_state_check'].fillna('-1')) 
        | (df['1'].fillna('-1') != df['payload_state_check'].fillna('-1')) 
        | (df['3'].fillna('-1') != df['launch_site_state_check'].fillna('-1')) 
        | (df['5'].fillna('-1') != df['lsp_state_check'].fillna('-1'))
           )

    if sum(check_filt) > 0 or len(df) != df_len:
        print('Something wrong with states df join')

    df = df.reset_index()


    # Extract date and time
    df['launch_datetime'] = df['launch_datetime'].str.split('[', expand=True)[0].str.split('(', expand=True)[0].str.strip()

    df['launch_datetime'] = df['launch_datetime'].str.strip()\
        .str.replace('1st Quarter', '1 January', regex=False)\
        .str.replace('2nd Quarter', '1 April', regex=False)\
        .str.replace('3rd Quarter', '1 July', regex=False)\
        .str.replace('4th Quarter', '1 October', regex=False)\
        .str.replace('?', '1 January', regex=False)\
        .str.replace('Late ', '', regex=False)\
        .str.replace('Early ', '', regex=False)

    only_month_filt = df['launch_datetime'].str.split(' ').str.len() == 1
    df.loc[only_month_filt, 'launch_datetime'] = '01 ' + df.loc[only_month_filt, 'launch_datetime'] 

    date_time = df['launch_datetime'].str.split(' ', expand=True)
    
    date_time[0] = date_time[0].str.strip().str.split('–', expand=True)[0]

    date_time[1] = date_time[1].str.replace('Unknown', 'January', regex=False).str.replace('Mid-', '', regex=False)
    
    try:
        date_time[2] = date_time[2].str.replace(r'[^0-9\:]+', '', regex=True).str.strip()
        date_time.loc[date_time[2].isin(['', ':']), 2] = None
    except:
        date_time[2] = None
    
    date_time.loc[date_time[2].isin(['', ':']), 2] = None
    date_time[3] = year

    try:
        date_time[4] = (date_time[2].str.split(':', expand=True)[0]).fillna('00').str.strip().str[:2]
    except:
        date_time[4] = '00'
    try:
        date_time[5] = (date_time[2].str.split(':', expand=True)[1]).fillna('00').str.strip().str[:2]
    except:
        date_time[5] = '00'
    try:
        date_time[6] = (date_time[2].str.split(':', expand=True)[2]).fillna('00').str.strip().str[:2]
    except:
        date_time[6] = '00'

    date = date_time[3] + ' ' + date_time[1] + ' ' + date_time[0]
    time = date_time[4] + ':' + date_time[5] + ':' + date_time[6]

    df['launch_datetime'] = pd.to_datetime(date + ' ' + time)


    # Create index for join rocket and payload
    filt = df['rocket'].notnull() | (df['rocket'].isnull() & (df['3'] == df['4']) & (df['5'] == df['6']))
    df.loc[~filt, 'index'] = None
    df['index'] = df['index'].fillna(method='ffill').astype(int)


    # Rocket df
    columns = ['index', 'launch_datetime', 'rocket', '1', '2', '3', '4', '5', '6', 'rocket_state', 'launch_site_state', 'lsp_state']
    df_rocket = df.loc[filt, columns].set_index('index')

    check_filt = (
        (df_rocket['rocket'].fillna('-1') != df_rocket['1'].fillna('-1'))
        | (df_rocket['3'].fillna('-1') != df_rocket['4'].fillna('-1'))
        | (df_rocket['5'].fillna('-1') != df_rocket['6'].fillna('-1'))
    )
    if sum(check_filt) > 0:
        print('Something wrong with df_rocket')

    df_rocket = df_rocket[['launch_datetime', 'rocket', 'rocket_state', '2', '3', 'launch_site_state', '5', 'lsp_state']]
    df_rocket.columns = ['launch_datetime', 'rocket', 'rocket_state', 'flight_number', 'launch_site', 'launch_site_state', 'lsp', 'lsp_state']


    # Payload df
    df_payload = df.loc[~filt, ['index', 'launch_datetime', '1', 'payload_state', '2', '3', '4', '5', '6']].set_index('index')
    df_payload.columns = ['payload_launch_datetime', 'payload', 'payload_state', 'payload_perator', 'payload_orbit', 'payload_function', 'payload_decay', 'payload_outcome']


    df_full = pd.concat([df_rocket, df_payload], axis=1) 

    check_filt = (df_full['launch_datetime'] != df_full['payload_launch_datetime']) 
    if sum(check_filt) > 0:
        print('Something wrong with df_rocket and df_payload join')
    df_full.drop('payload_launch_datetime', axis=1, inplace=True)

    for column in df_full.columns:
        if column != 'launch_datetime':
            df_full[column] = (df_full[column].str.split('[', expand=True)[0]).fillna('').str.replace('?', '', regex=False).str.strip()
            df_full.loc[df_full[column] == '', column] = None

    return df_full.reset_index()

In [3]:
response = requests.get('https://en.wikipedia.org/wiki/Category:Spaceflight_by_year')
soup = BeautifulSoup(response.text, 'html.parser')
all_a = soup.find_all('a')

urls = []

for a in all_a:
    url = a.get('href')
    if url and 'in_spaceflight' in url and 'Category' not in url:
        urls.append(url)
    elif url and 'List_of_spaceflight_launches' in url and 'Category' not in url:
        urls.append(url)

first_run = True
year = '0'


for url in urls: 
    print(url)
    
    if 'in_spaceflight' in url:
        new_year = url[6:10]
    elif 'List_of_spaceflight_launches' in url:
        new_year = url[-4:]
        
    if year != new_year:
        n = 1
        
    year = new_year
    
    if int(year) > 2023:
        continue
    
    response = requests.get('https://en.wikipedia.org' + url)

    soup = BeautifulSoup(response.text, 'html.parser')
    all_tables = soup.find_all('table', {'class':"wikitable"})
    
    tables = []
    
    for table in all_tables:
        column_name = table.find('th', {'rowspan': 3})
        if len(table.text) > 178 and column_name and column_name.text.strip() == 'Date and time (UTC)':
            tables.append(table)
    
    print(len(tables))
    
    for table in tables:
        print(n)
        if first_run:
            df = get_data(table, year)
            df['year'] = int(year)
            df['table_n'] = n
            first_run = False
        else:
            temp_df = get_data(table, year)
            temp_df['year'] = int(year)
            temp_df['table_n'] = n
            df = pd.concat([df, temp_df], ignore_index=True)
            
        
        n += 1

/wiki/2020s_in_spaceflight
0
/wiki/1952_in_spaceflight
10
1
2
3
4
5
6
7
8
9
10
/wiki/1953_in_spaceflight
11
1
2
3
4
5
6
7
8
9
10
11
/wiki/1954_in_spaceflight
11
1
2
3
4
5
6
7
8
9
10
11
/wiki/1955_in_spaceflight
12
1
2
3
4
5
6
7
8
9
10
11
12
/wiki/1956_in_spaceflight
12
1
2
3
4
5
6
7
8
9
10
11
12
/wiki/1957_in_spaceflight
13
1
2
3
4
5
6
7
8
9
10
11
12
13
/wiki/1958_in_spaceflight
13
1
2
3
4
5
6
7
8
9
10
11
12
13
/wiki/1959_in_spaceflight
13
1
2
3
4
5
6
7
8
9
10
11
12
13
/wiki/1960_in_spaceflight
0
/wiki/1960_in_spaceflight_(January%E2%80%93June)
2
1
2
/wiki/1960_in_spaceflight_(July%E2%80%93December)
2
3
4
/wiki/1961_in_spaceflight
0
/wiki/1961_in_spaceflight_(January%E2%80%93June)
2
1
2
/wiki/1961_in_spaceflight_(July%E2%80%93December)
2
3
4
/wiki/1962_in_spaceflight
0
/wiki/1962_in_spaceflight_(January%E2%80%93March)
2
1
2
/wiki/1962_in_spaceflight_(April%E2%80%93June)
2
3
4
/wiki/1962_in_spaceflight_(July%E2%80%93September)
2
5
6
/wiki/1962_in_spaceflight_(October%E2%80%93December)
2

In [4]:
df

,index,launch_datetime,rocket,rocket_state,flight_number,launch_site,launch_site_state,lsp,lsp_state,payload,payload_state,payload_perator,payload_orbit,payload_function,payload_decay,payload_outcome,year,table_n
0,0,1952-01-30 20:45:00,Aerobee RTV-A-1a,United States,USAF 21,Holloman LC-A,United States,US Air Force,United States,Ionosphere 1,None,AFCRC / University of Utah,Suborbital,Ionospheric,30 January,Launch failure,1952,1
1,0,1952-02-19 14:49:00,Aerobee RTV-A-1c,United States,USAF 22,Holloman LC-A,United States,US Air Force,United States,None,None,AFCRC / University of Utah,Suborbital,Airglow,19 February,Launch failure,1952,2
2,4,1952-02-19 17:00:00,Aerobee RTV-N-10,United States,NRL 7,White Sands LC-35,United States,US Navy,United States,None,None,NRL,Suborbital,Cosmic Radiation / Solar Radiation,19 February,Successful,1952,2
3,8,1952-02-29 14:40:00,Aerobee RTV-A-1,United States,USAF 23,Holloman LC-A,United States,US Air Force,United States,None,None,AFCRC / University of Utah,Suborbital,Airglow,29 February,Successful,1952,2
4,0,1952-04-22 17:28:00,Aerobee RTV-A-1,United States,USAF 24,Holloman LC-A,United States,US Air Force,United States,None,None,AFCRC / Boston University,Suborbital,Ionospheric,22 April,Successful,1952,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16973,149,2023-12-11 14:38:00,IRBM-T1,United States,FTG-12,"Boeing C-17 Globemaster III, Pacific Ocean",United States,MDA,United States,None,United States,MDA,Suborbital,Missile target,11 December,Successful,2023,4
16974,153,2023-12-11 14:48:00,GBI-OBV,United States,FTG-12,Vandenberg LF-24,United States,MDA,United States,None,United States,MDA,Suborbital,Interceptor,11 December,Successful,2023,4
16975,157,2023-12-17 23:24:00,Hwasong-18,North Korea,None,Sunan,North Korea,KPA Strategic Force,North Korea,None,North Korea,KPA Strategic Force,Suborbital,Missile test,17 December,Successful,2023,4
16976,161,2023-12-19 16:42:28,New Shepard,United States,NS-24,Corn Ranch,United States,Blue Origin,United States,Blue Origin NS-24,United States,Blue Origin,Suborbital,Uncrewed commercial spaceflight,19 December 16:52:41,Successful,2023,4


In [5]:
df

,index,launch_datetime,rocket,rocket_state,flight_number,launch_site,launch_site_state,lsp,lsp_state,payload,payload_state,payload_perator,payload_orbit,payload_function,payload_decay,payload_outcome,year,table_n
0,0,1952-01-30 20:45:00,Aerobee RTV-A-1a,United States,USAF 21,Holloman LC-A,United States,US Air Force,United States,Ionosphere 1,None,AFCRC / University of Utah,Suborbital,Ionospheric,30 January,Launch failure,1952,1
1,0,1952-02-19 14:49:00,Aerobee RTV-A-1c,United States,USAF 22,Holloman LC-A,United States,US Air Force,United States,None,None,AFCRC / University of Utah,Suborbital,Airglow,19 February,Launch failure,1952,2
2,4,1952-02-19 17:00:00,Aerobee RTV-N-10,United States,NRL 7,White Sands LC-35,United States,US Navy,United States,None,None,NRL,Suborbital,Cosmic Radiation / Solar Radiation,19 February,Successful,1952,2
3,8,1952-02-29 14:40:00,Aerobee RTV-A-1,United States,USAF 23,Holloman LC-A,United States,US Air Force,United States,None,None,AFCRC / University of Utah,Suborbital,Airglow,29 February,Successful,1952,2
4,0,1952-04-22 17:28:00,Aerobee RTV-A-1,United States,USAF 24,Holloman LC-A,United States,US Air Force,United States,None,None,AFCRC / Boston University,Suborbital,Ionospheric,22 April,Successful,1952,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16973,149,2023-12-11 14:38:00,IRBM-T1,United States,FTG-12,"Boeing C-17 Globemaster III, Pacific Ocean",United States,MDA,United States,None,United States,MDA,Suborbital,Missile target,11 December,Successful,2023,4
16974,153,2023-12-11 14:48:00,GBI-OBV,United States,FTG-12,Vandenberg LF-24,United States,MDA,United States,None,United States,MDA,Suborbital,Interceptor,11 December,Successful,2023,4
16975,157,2023-12-17 23:24:00,Hwasong-18,North Korea,None,Sunan,North Korea,KPA Strategic Force,North Korea,None,North Korea,KPA Strategic Force,Suborbital,Missile test,17 December,Successful,2023,4
16976,161,2023-12-19 16:42:28,New Shepard,United States,NS-24,Corn Ranch,United States,Blue Origin,United States,Blue Origin NS-24,United States,Blue Origin,Suborbital,Uncrewed commercial spaceflight,19 December 16:52:41,Successful,2023,4


In [6]:
df['flight_n'] = (df['year'] * 1000000 + df['table_n'] * 10000 +  df['index']).rank(method='dense').astype(int)

In [7]:
df.tail(30)

,index,launch_datetime,rocket,rocket_state,flight_number,launch_site,launch_site_state,lsp,lsp_state,payload,payload_state,payload_perator,payload_orbit,payload_function,payload_decay,payload_outcome,year,table_n,flight_n
16948,65,2023-10-18 00:00:00,Ababeel,Pakistan,None,None,Pakistan,Pakistan Army,Pakistan,None,Pakistan,Pakistan Army,Suborbital,Missile test,18 October,Successful,2023,4,12925
16949,68,2023-10-24 00:00:00,Ghauri,Pakistan,None,None,Pakistan,Pakistan Army,Pakistan,None,Pakistan,Pakistan Army,Suborbital,Missile test,24 October,Successful,2023,4,12926
16950,71,2023-10-25 00:00:00,Terrier Oriole,United States,FTM-48,Barking Sands,United States,United States Navy,United States,None,United States,United States Navy,Suborbital,Missile target,25 October,Successful,2023,4,12927
16951,75,2023-10-25 00:00:00,Terrier Oriole,United States,FTM-48,Barking Sands,United States,United States Navy,United States,None,United States,United States Navy,Suborbital,Missile target,25 October,Successful,2023,4,12928
16952,79,2023-10-25 00:00:00,SM-3 Block IA,United States,FTM-48,"USS Carl M. Levin, Pacific Ocean",United States,United States Navy,United States,Kill vehicle,United States,United States Navy,Suborbital,Interceptor,25 October,Successful,2023,4,12929
16953,83,2023-10-25 00:00:00,SM-3 Block IA,United States,FTM-48,"USS Carl M. Levin, Pacific Ocean",United States,United States Navy,United States,Kill vehicle,United States,United States Navy,Suborbital,Interceptor,25 October,Successful,2023,4,12930
16954,87,2023-10-25 00:00:00,RS-24 Yars,Russia,None,None,Russia,Russian Ministry of Defence,Russia,None,Russia,Russian Ministry of Defence,Suborbital,ICBM test,25 October,Successful,2023,4,12931
16955,90,2023-10-25 00:00:00,RSM-54 Sineva,Russia,None,"K-114 Tula, Barents Sea",Russia,Russian Ministry of Defence,Russia,None,Russia,Russian Ministry of Defence,Suborbital,SLBM test,25 October,Successful,2023,4,12932
16956,94,2023-10-30 05:45:00,Black Brant IX,Canada,None,White Sands Missile Range,United States,NASA,United States,INFUSE,United States,CU Boulder,Suborbital,Ultraviolet astronomy,30 October,Successful,2023,4,12933
16957,98,2023-10-31 00:00:00,Qader,Yemen,None,None,Yemen,Houthis,Yemen,Live warhead,Yemen,Houthis,Suborbital,Missile launch,31 October,Intercepted,2023,4,12934


In [8]:
df.to_csv('space_launces.csv', index=False)